In [1]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"));

import sys
sys.path.append('..')
from LocationsDB import LocationsDatabase

from datetime import datetime

from glob import glob

In [2]:
loc_db = LocationsDatabase('../database_files/ECON_Locations_DB.db')
# loc_db = LocationsDatabase('../database_files/testing_database.db')

In [3]:
### tray 72 short by 16 chips

data = []
for tray in range(22,79):
    lot_,date_,timestamp_ = 'N61.H52.00','2024/29',datetime.strptime('2024-08-15','%Y-%m-%d')
    N=90
    if tray==72:
        N=74
    for chip in range(1,N+1):
        chip_id = tray*100 + chip
        data.append([chip_id,tray,chip,lot_,date_,timestamp_])

In [4]:
for d in data:
    chip_id,tray,pos,pkg_batch,pkg_date,timestamp = d
    loc_db.checkin_chip(chip_id,'ECONT',tray,pos,'WH14',pkg_date,pkg_batch,'',str(timestamp))
loc_db.conn.commit()    

In [5]:
loc_db.loadLocationsDatabase().shape

(32498, 9)

In [6]:
loc_db.loadStatusDatabase().shape

(17712, 7)

In [7]:
testing_df = pd.read_csv('../testingCSV/econt_chip_test_results_2025-04-24_09-25-14.csv')


c = list(testing_df.columns)
c[0] = 'chip_id'
testing_df.columns = c

testing_df = testing_df[['chip_id','Timestamp']]

testing_df.Timestamp = pd.to_datetime(testing_df.Timestamp)
testing_df.sort_values('Timestamp',inplace=True)

testing_df.reset_index(inplace=True,drop=True)

testing_df.dropna(inplace=True)


In [8]:
d = loc_db.getCurrentLocations()
d.set_index('chip_id',inplace=True)
for x in testing_df.values:
    chip_id,time = x
    if len(loc_db.getChip(chip_id))<1:
        print(f'Chip {chip_id} not in database, tested {time}')
        continue
    _tray,_pos = d.loc[chip_id][['current_tray','current_position']].values.flatten()
    loc_db.setTestedStatus(chip_id,_tray,_pos,'',str(time))
loc_db.conn.commit()

Chip 1601 not in database, tested 2024-11-22 18:07:28
Chip 1202 not in database, tested 2024-11-22 18:10:58
Chip 1602 not in database, tested 2024-11-22 18:14:19
Chip 1203 not in database, tested 2024-11-22 18:17:08
Chip 1603 not in database, tested 2024-11-22 18:20:27
Chip 1204 not in database, tested 2024-11-22 18:23:09
Chip 1604 not in database, tested 2024-11-22 18:26:28
Chip 1205 not in database, tested 2024-11-22 18:29:11
Chip 1605 not in database, tested 2024-11-22 18:33:16
Chip 1206 not in database, tested 2024-11-22 18:35:58
Chip 1606 not in database, tested 2024-11-22 18:39:20
Chip 1207 not in database, tested 2024-11-22 18:42:06
Chip 1208 not in database, tested 2024-11-22 18:48:15
Chip 1209 not in database, tested 2024-11-22 18:53:38
Chip 1210 not in database, tested 2024-11-22 18:58:56
Chip 1211 not in database, tested 2024-11-22 19:04:17
Chip 1212 not in database, tested 2024-11-22 19:09:36
Chip 1213 not in database, tested 2024-11-22 19:14:57
Chip 1214 not in database, t

Chip 1352 not in database, tested 2024-11-23 23:08:34
Chip 1353 not in database, tested 2024-11-23 23:13:43
Chip 1354 not in database, tested 2024-11-23 23:18:54
Chip 1355 not in database, tested 2024-11-23 23:24:07
Chip 1356 not in database, tested 2024-11-23 23:29:16
Chip 1358 not in database, tested 2024-11-23 23:39:32
Chip 1359 not in database, tested 2024-11-23 23:44:41
Chip 1360 not in database, tested 2024-11-23 23:49:51
Chip 1361 not in database, tested 2024-11-23 23:55:05
Chip 1362 not in database, tested 2024-11-24 00:01:04
Chip 1363 not in database, tested 2024-11-24 00:06:15
Chip 1364 not in database, tested 2024-11-24 00:11:24
Chip 1365 not in database, tested 2024-11-24 00:16:34
Chip 1366 not in database, tested 2024-11-24 00:21:44
Chip 1367 not in database, tested 2024-11-24 00:26:59
Chip 1368 not in database, tested 2024-11-24 00:32:09
Chip 1369 not in database, tested 2024-11-24 00:37:18
Chip 1370 not in database, tested 2024-11-24 00:42:28
Chip 1371 not in database, t

In [9]:
loc_db.loadLocationsDatabase().shape

(37515, 9)

In [10]:
loc_db.loadStatusDatabase().shape

(17712, 7)

In [11]:
d_ = []
for f in glob('../sorting_logs/ECONT_*'):
    d = pd.read_csv(f,header=None)
    d.columns = ['start_tray','start_pos','end_tray','end_pos']
    d['timestamp'] = datetime.strptime(f.split('_')[2],'%Y%m%d')
    if 'Nadja' in f:
        d['shippingLocation'] = 'UMN'
    if 'Paul' in f:
        d['shippingLocation'] = 'Baylor-Partial'### CMs get listed as Baylor for final location
    
    d_.append(d)
sorting_df = pd.concat(d_)
sorting_df['start_tray_int'] = sorting_df.start_tray.str[-5:].astype(int)

### CORRECT THE TRAY NUMBERS TO PUT THEM INTO THE APPROPRIATE SPOT IN THE TRAY SEQUENCE
sorting_df['end_tray_corrected'] = sorting_df['end_tray'].str.replace('00801','08009').str.replace('00802','08010').str.replace('00803','08011').str.replace('00804','08012')
sorting_df['end_tray_int'] = sorting_df.end_tray_corrected.str[-5:].astype(int)

sorting_df['chip_id'] = sorting_df.start_tray_int*100 + sorting_df.start_pos

sorting_df.reset_index(drop=True,inplace=True)


for idx in sorting_df.index:
    s = sorting_df.loc[idx]
    chip_id = s.chip_id
    if not 'TESTED' in loc_db.getChip(chip_id).entry_type.values:
        print(f'ISSUE WITH {chip_id}')
    else:
        loc_db.sortChip(int(chip_id),int(s.start_tray_int),int(s.start_pos),int(s.end_tray_int),int(s.end_pos),'',str(s.timestamp))
        loc_db.setChipGrade(int(chip_id),'PRESERIES','',str(s.timestamp))
loc_db.conn.commit()

In [12]:
loc_db.loadLocationsDatabase().shape

(37755, 9)

In [13]:
loc_db.loadStatusDatabase().shape

(17952, 7)

In [14]:
loc_db.getChipsInTray(8009)

,chip_id,entry_type,initial_tray,initial_position,current_tray,current_position,location,comments,time
37649,2201,SORTED,22,1,8009,15,WH14,,2025-01-30
37673,2203,SORTED,22,3,8009,39,WH14,,2025-01-30
37661,2204,SORTED,22,4,8009,27,WH14,,2025-01-30
37682,2205,SORTED,22,5,8009,48,WH14,,2025-01-30
37678,2206,SORTED,22,6,8009,44,WH14,,2025-01-30
...,...,...,...,...,...,...,...,...,...
37721,2370,SORTED,23,70,8009,87,WH14,,2025-01-30
37718,2375,SORTED,23,75,8009,84,WH14,,2025-01-30
37717,2379,SORTED,23,79,8009,83,WH14,,2025-01-30
37723,2380,SORTED,23,80,8009,89,WH14,,2025-01-30


In [15]:
loc_db.getChipsInTray(22)

,chip_id,entry_type,initial_tray,initial_position,current_tray,current_position,location,comments,time
32500,2202,TESTED,22,2,22,2,WH14,,2025-01-21 16:09:29
32531,2218,TESTED,22,18,22,18,WH14,,2025-01-21 18:11:21
32533,2219,TESTED,22,19,22,19,WH14,,2025-01-21 18:17:56
32537,2221,TESTED,22,21,22,21,WH14,,2025-01-21 18:30:46
32545,2225,TESTED,22,25,22,25,WH14,,2025-01-21 18:57:29
32613,2259,TESTED,22,59,22,59,WH14,,2025-01-21 22:37:58
32618,2263,TESTED,22,63,22,63,WH14,,2025-01-21 23:03:13
32629,2274,TESTED,22,74,22,74,WH14,,2025-01-22 00:03:18
32631,2276,TESTED,22,76,22,76,WH14,,2025-01-22 00:34:35
32632,2277,TESTED,22,77,22,77,WH14,,2025-01-22 00:41:04
